This is my First Try in a Competition so Feedback would be appreciated!

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import  cross_val_score,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB,ComplementNB,MultinomialNB

In [ ]:

data = pd.read_csv("../input/entrepreneurial-competency-in-university-students/data.csv")

#Seperate Label From Data
label = data["y"]

#Seperate the Features from the Data
features = data.drop(columns=[ "Age","y","ReasonsForLack"])


columns = features.columns

#Seperate the Situational Data from the survey Data
meta_data = ["EducationSector", "Gender", "City"]
self_reflection = ["IndividualProject","Influenced","Perseverance","DesireToTakeInitiative","Competitiveness","SelfReliance","StrongNeedToAchieve","SelfConfidence","GoodPhysicalHealth","KeyTraits"]
data.head(10)

I decided to Split the Data into 2 Types. The First one is Metadata which is everything that can not be freely chosen by the subject, the other one is self_reflection which cosists of the self estimation of the subject. The Reason for this spilt is that certain Educational Sectors,Ages and Genders have a different probablility to be Entrepreneurial than others. I will use a Naive Bayes Net to Calculate the Probabiltiy of the subject being Entrepreneurial based on their Meta_Data.

After doing that i will Create a RandomizedSearchCV to find the best Prams for the Tree and then use cross_val_score to score the Forest.

In [ ]:
#Visualize the Distribution of Data
for col in data:
    ax= None
    libdata= data[col].value_counts()
    #print(data[col].value_counts() )
    #print(data[col].unique())
    print("Col: "+ col)
    ax = sns.barplot(x = libdata.index , y=libdata.values )
    plt.show()


#data["EducationSector"].groupby().count



In [ ]:
#Encoding string Features witha with a OrdinalEncoder
enc = OrdinalEncoder()
features_formated = pd.DataFrame(enc.fit_transform(features[columns]))
features.drop(columns=columns)
features[columns] = features_formated
features.head(10)

I am using a OrdinalEncoder to Change string Data into Floats

In [ ]:
#Use a Gaussian NB to Predict from the Probability for Enterpreneurial Competency from the Metadata
classifier  = BernoulliNB()
score = cross_val_score(classifier, features[meta_data], label)
print(score.mean())
classifier.fit(features[meta_data],label)
proba = pd.DataFrame(classifier.predict_proba(features[meta_data]))
features["BernoulliNB"] = proba[1]
columns = features.columns
features.head(20)
self_reflection.append("BernoulliNB")

Predicting the probability for every Dataset to be an Enterpreneur

In [ ]:

#Try to get the Optimal Parameters for a RandomforestClassifier
classifier = RandomForestClassifier()
dit = classifier.get_params(deep=True)

dit= param_dist = {"max_depth": [25,50,100,150,200, None],
                    "n_estimators": [50,100,120,150,200],
                "criterion": ["gini", "entropy"],
              "max_features": [1,2,3, "auto", "sqrt", "log2"],
              "min_samples_leaf": [1,2,3,4,5,6,7,8,9],
                   "min_impurity_decrease" : [0.0,0.2,0.3,0.5,0.7],
                   "ccp_alpha" : [0.0,0.2,0.4,0.6]
             }
clf = RandomizedSearchCV(classifier, dit, random_state=0, cv=3)
search = clf.fit(features[self_reflection],label)
params = search.best_params_
print(params)

I use an RandomizedSearchCV to tune the Hyperparameters for the best accuracy over 3 folds

In [ ]:
#Use A RandomeForestClassifier with the Optimal params in combinaiton with the Meta-Prediction to predict the final label
pltY=[]
classifier = RandomForestClassifier(**params)
score = cross_val_score(classifier,features[self_reflection],label, cv=20)
print("%0.2f accuracy with a standard deviation of %0.2f" % (score.mean(), score.std()))

#AS we can See, the Accuracy varies widely between 0.8 and 0.5
plt.plot(np.arange(1,len(score)+1),score)
plt.show()

**Conclusion**:
We can see that the fluctuation of the Accuracy is relativly high for only 20 folds. What is also interesing is that in some splits an accuracy of 0.9 is accieved and on other splits it goes as low as 0.5. Sadly I no clue how i would be able to stabelize the accuracy over 0.75.

I would really appreciate Feedback for this Notebook.

Addendum: Maybe I will add more comments regarding my actions and conclusions later